1KG
==
Run mushi on 3-SFS computed from 1000 Genome Project data

In [ ]:
%matplotlib inline 
#%matplotlib notebook
import histories
import mushi
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.special import expit
import time
import msprime
%cd stdpopsim
from stdpopsim import homo_sapiens
%cd ../

### Load 1KG 3-SFS

In [ ]:
ksfs_CEU = pd.read_csv('1KG/scons_output/3-SFS.tsv', sep='\t', index_col=0)

mutation_types = ksfs_CEU.columns
ksfs_CEU

Rank plot of the number of SNPs of each triplet mutation type

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(ksfs_CEU.sum().sort_values(ascending=False).to_frame(), '.')
plt.xticks(rotation='vertical', family='monospace')
plt.ylabel('number of SNPs')
plt.yscale('symlog')
plt.tight_layout()
plt.savefig('/Users/williamdewitt/Downloads/foo.png')
plt.show()

In [ ]:
# plt.style.use('dark_background')

### Demographic history $\eta(t)$ from the European population in Browning et al.
Browning, S. R. et al. Ancestry-specific recent effective population size in the Americas. [PLOS Genetics 14, e1007385 (2018)](https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1007385).

In [ ]:
t = np.logspace(0, np.log10(3e4), 500)

In [ ]:
model = homo_sapiens.BrowningAmerica()
dd = msprime.DemographyDebugger(Ne=model.default_population_size,
                                population_configurations=model.population_configurations,
                                demographic_events=model.demographic_events,
                                migration_matrix=model.migration_matrix)
y = 2 * dd.population_size_trajectory(np.concatenate(([0], t)))[:, 1]
η = histories.η(t, y)

plt.figure(figsize=(3, 3))
η.plot()
plt.show()

### Mushi $k$-SFS object conditioned on this demographic history

In [ ]:
ksfs = mushi.kSFS(η, X=ksfs_CEU.values, mutation_types=mutation_types)

### TMRCA CDF

In [ ]:
plt.figure(figsize=(3, 3))
plt.plot(η.change_points, ksfs.tmrca_cdf())
plt.xlabel('$t$')
plt.ylabel('TMRCA CDF')
plt.ylim([0, 1])
plt.xscale('symlog')
plt.tight_layout()
plt.show()

### Mutation type enrichment traces, as in Harris and Pritchard

In [ ]:
plt.figure(figsize=(4, 3))
foo = ksfs_CEU / ksfs_CEU.values.sum(1, keepdims=True)
# foo = foo / foo.mean(0)
plt.plot(foo, c='k', lw=1, alpha=0.05)
plt.plot(foo['TCC>TTC'], label='TCC>TTC', c='C0', lw=2, alpha=0.8)
plt.xlabel('sample frequency')
plt.ylabel('mutation type fraction')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.savefig('/Users/williamdewitt/Downloads/X_twiggle.pdf')
plt.show()

### Mutation type enrichment again, but as a heatmap with correlation clustering

In [ ]:
ksfs.clustermap(figsize=(25, 10))
plt.show()

### Invert the $k$-SFS conditioned on $\eta(t)$ to get $\boldsymbol\mu(t)$
Accelerated proximal gradient descent

In [ ]:
μ_inferred, f_trajectory = ksfs.infer_μ(# loss function parameters
                                        fit='prf',
                                        exclude_singletons=False,
                                        bins=None,
                                        # time derivative regularization parameters
                                        λ_tv=1e2,
                                        α_tv=0,
                                        # spectral regularization parameters
                                        λ_r=1e4,
                                        α_r=.9/1e4,
                                        hard=True,                                        
                                        # convergence parameters
                                        max_iter=1000,
                                        tol=1e-10,
                                        γ=0.8)

Convergence

In [ ]:
plt.figure(figsize=(4, 2))
plt.plot(f_trajectory)
plt.xlabel('iterations')
plt.ylabel('loss')
plt.xscale('symlog')
plt.tight_layout()
plt.show()

The inferred histories for each mutation type (raw mutation rate in units of mutations per genome per generation)

In [ ]:
plt.figure(figsize=(6, 3))
μ_inferred.plot(alpha=0.1, lw=2)
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
foo = μ_inferred.Z / μ_inferred.Z.sum(1, keepdims=True)
foo = foo / foo.mean(0)
plt.plot(foo, c='k', lw=1, alpha=0.05)
plt.plot(foo[:, μ_inferred.mutation_types.get_loc('TCC>TTC')], label='TCC>TTC', c='C0', lw=2, alpha=0.8)
plt.xlabel('$t$')
plt.ylabel('mutation type enrichment')
plt.xscale('log')
plt.legend()
plt.tight_layout()
plt.show()

Heatmap of the inferred mutation spectrum history, plotted as relative mutation intensity as in Harris and Pritchard

In [ ]:
μ_inferred.clustermap(figsize=(25, 10))
plt.show()

plot $\chi^2$ goodness of fit for each $k$-SFS matrix element, and compute $\chi^2$ goodness of fit test for the $k$-SFS matrix as a whole

In [ ]:
ksfs.clustermap(μ_inferred, figsize=(25, 10), cmap='Reds')
plt.show()

Plot SFS fit for the first 10 mutation types individually

In [ ]:
f, axes = plt.subplots(10, 1, sharex=True, figsize=(3, 10))
for ct, i in enumerate(range(10)):
    plt.sca(axes[ct])
    ksfs.plot(i, μ=μ_inferred, prf_quantiles=True)
    plt.ylabel(μ_inferred.mutation_types[ct])
    if ct < 9:
        plt.xlabel(None)
plt.tight_layout()
plt.show()

### Singular value spectrum of $Z$

In [ ]:
plt.figure(figsize=(3, 3))
plt.bar(range(μ_inferred.Z.shape[1]), np.linalg.svd(μ_inferred.Z, compute_uv=False))
plt.yscale('log')
plt.tight_layout()
plt.show()